# Move files exported by Equivital Qiosk

Check and clear 

Plot and review measurements from equivital sensors as output by the Equivital Manager app.
(Every program produces different formated files and it's a pain.)

In [1]:
import sys
import os
import shutil
import time
import datetime as dt
import math
import numpy as np 
import scipy as sp
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
# import heartpy as hp

from scipy.signal import butter,filtfilt
from scipy import interpolate
from scipy.interpolate import interp1d

## Definitions for data extraction

In [2]:
%load_ext autoreload
%autoreload 1
%aimport qex

In [77]:
%reload_ext autoreload

In [4]:
seperator = str('\\')
seperator

'\\'

# Scan for DATA files

All recordinds exported by Qiosk produce four CSV files and one SEM file. One of the CSVs is a DATA file that contains metadata and some essential signal quality checks on 15 s intervals.

We can ID instances of signal recordings from statistics taken out of the DATA file, so that gets extracted first

In [12]:
# location of the files when exported by QIOSK
path = "C:\\Users\\Public\\Documents\\Equivital\\Equivital Manager Wizard\\"#Extracted SEM Data\\" #"../Equivital/Equivital Manager Wizard/Extracted SEM Data/"
#os.listdir(path)

df_datafiles = qex.qiosk_recordings(path,'Present',seperator)
df_datafiles

,Signal,DevName,ID,Date,Session,FileName,FileType,FileSize,RecStart,RecEnd,...,SubjectNames,SENSOR ID,SUBJECT ID,SUBJECT AGE,HR(BPM),HRC(%),BELT OFF,LEAD OFF,MOTION,BODY POSITION
0,DATA,Pilot 58,5022240,230130,0,DATA-Pilot 58-5022240-23013000.CSV,csv,219,2023-01-30 14:55:15+00:00,2023-01-30 14:55:15+00:00,...,[Pilot 58],5022240.0,Pilot 58,NaN,130.0,67.0,1.0,1.0,Stationary,Prone
1,DATA,Pilot 57,5022002,230130,0,DATA-Pilot 57-5022002-23013000.CSV,csv,221,2023-01-30 14:55:15+00:00,2023-01-30 14:55:15+00:00,...,[Pilot 57],5022002.0,Pilot 57,NaN,56.0,10.0,1.0,1.0,Stationary,Inverted
2,DATA,Pilot 42,5022143,230131,0,DATA-Pilot 42-5022143-23013100.CSV,csv,219,2023-01-31 10:22:56+00:00,2023-01-31 10:22:56+00:00,...,[Pilot 42],5022143.0,Pilot 42,NaN,104.0,95.0,1.0,1.0,Stationary,Prone
3,DATA,Pilot_1,3420755,230206,0,DATA-Pilot_1-3420755-23020600.CSV,csv,188,2023-02-06 10:35:11+00:00,2023-02-06 10:35:11+00:00,...,[Pilot_1],3420755.0,Pilot_1,NaN,6.0,100.0,1.0,1.0,MovingSlowly,Prone
4,DATA,Pilot_5,5022126,230207,0,DATA-Pilot_5-5022126-23020700.CSV,csv,185,2023-02-07 10:09:13+00:00,2023-02-07 10:09:13+00:00,...,[Pilot_5],5022126.0,Pilot_5,NaN,32.0,81.0,1.0,1.0,Stationary,Side
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
74,DATA,VN202,5022051,230214,1,DATA-VN202-5022051-23021401.CSV,csv,545,2023-02-14 13:46:06+00:00,2023-02-14 13:47:06+00:00,...,[VN202],5022051.0,VN202,NaN,92.0,100.0,1.0,1.0,Stationary,Side
75,DATA,VN207,5022073,230214,2,DATA-VN207-5022073-23021402.CSV,csv,944,2023-02-14 13:46:08+00:00,2023-02-14 13:48:23+00:00,...,[VN207],5022073.0,VN207,NaN,122.0,71.0,1.0,1.0,Stationary,Side
76,DATA,VN203,5022057,230214,1,DATA-VN203-5022057-23021401.CSV,csv,546,2023-02-14 13:46:10+00:00,2023-02-14 13:47:10+00:00,...,[VN203],5022057.0,VN203,NaN,120.0,100.0,1.0,1.0,Stationary,Side
77,DATA,Bluetooth04,5022297,230214,1,DATA-Bluetooth04-5022297-23021401.CSV,csv,1914,2023-02-14 15:11:53+00:00,2023-02-14 15:16:38+00:00,...,[Bluetooth04],5022297.0,Bluetooth04,NaN,64.0,38.0,0.0,1.0,MovingSlowly,Upright


In [13]:
print('Number of DATA files: '+ str(len(df_datafiles)))
# key categories to structure of Qiosk output DATA files
print(df_datafiles['ID'].unique())
print(df_datafiles['DevName'].unique())
print(df_datafiles['Date'].unique())

Number of DATA files: 79
[5022240 5022002 5022143 3420755 5022126 3420415 5022207 5022198 5022100
 5022073 5022285 5022281 5022297 5022292 5022050 5022179 5022138 5022139
 5022184 5022193 5022101 5022104 5022238 5022051 3420426 5022057 5022141
 5022142 5022117 5022241 5022110 5022155 3420427 5023926 5022156 5022060
 5022149 5022161 5022058 5022077 5022154 3420456 5022173 5022245 5022163
 5022164 5022062 5022127 5022202 5022203 5022076 5022204 5022098 5022131
 5022151 5022166 5022152 5022006]
['Pilot 58' 'Pilot 57' 'Pilot 42' 'Pilot_1' 'Pilot_5' 'Bluetooth_Tester_1'
 'BR608' 'BR604' 'VA301' 'VN207' 'Bluetooth02' 'Bluetooth01' 'Bluetooth04'
 'Bluetooth03' 'VN201' 'BR601' 'VN101' 'LS401' 'LS402' 'BR602' 'BR603'
 'VA302' 'VA303' 'PC701' 'PC702' 'VN202' 'VN102' 'VN203' 'LS403' 'LS404'
 'VA305' 'PC703' 'VA304' 'WW501' 'VN105' 'VN103' 'LS406' 'WW502' 'VN205'
 'LS407' 'LS405' 'WW503' 'VN204' 'VN209' 'LS410' 'VA306' 'VN104' 'WW507'
 'PC705' 'WW504' 'WW505' 'VN206' 'VA307' 'BR605' 'BR606' 'VN208

## Define set of recordings to move
First specify a subset by either device number (singular) or day of recording event.

Next this subset can be refined by restricting recordings to a specific time of day: to have started before a particular time (when you know the recording was happening) and/or to have ended after a specific time (when you know the recording was happening.

All times are in UTC, which is an hour earlier than Central European Time, 2 hours earlier than Central European Summer Time.


In [86]:
# all the recordings that are in the directiory
s_files = df_datafiles

In [66]:
# DATA files from one device, by device ID numbr
devN = 3420456
s_files=df_datafiles.loc[df_datafiles['ID'] == devN,:] 

In [164]:
# DATA files from recordings on a particular day, YYMMDD
recDate = 230201
s_files=df_datafiles.loc[df_datafiles['Date'] == recDate,:]


In [9]:
# DATA files from recordings that lasted longer than N seconds
minDuration = 600  
s_files=df_datafiles.loc[df_datafiles['Duration'] > minDuration,:]


In [7]:
# DATA files from recordings that are smaller than N Bytes
maxfileSize = 150 
s_files=df_datafiles.loc[df_datafiles['FileSize'] < maxfileSize,:]


Additionally select according to start and stop recording times.

In [328]:
# recordings started before a certain time 
t = pd.to_datetime('2023-02-01 12:02:00.00+0000')# V['DateTime'].iloc[int(len(V)/2)]   2023-01-18 13:30:09 2023-01-18 13:43:27
s_files=s_files.loc[s_files['RecStart']< t,:] 

In [68]:
# recordings ended after a certain time
t = pd.to_datetime('2023-01-28 00:00:00.00+0000')# V['DateTime'].iloc[int(len(V)/2)]   2023-01-18 13:30:09 2023-01-18 13:43:27
s_files=s_files.loc[s_files['RecEnd']> t,:] 

In [8]:
# show subset of files
s_files

,Signal,DevName,ID,Date,Session,FileName,FileType,FileSize,RecStart,FullLoc,...,SubjectNames,SENSOR ID,SUBJECT ID,SUBJECT AGE,HR(BPM),HRC(%),BELT OFF,LEAD OFF,MOTION,BODY POSITION
0,DATA,Bluetooth01,5022281,230213,0,DATA-Bluetooth01-5022281-23021300.CSV,csv,134,2020-02-02 02:02:00+00:00,C:\Users\Public\Documents\Equivital\Equivital ...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,DATA,Pilot_3,3420427,230130,0,DATA-Pilot_3-3420427-23013000.CSV,csv,109,2020-02-02 02:02:00+00:00,C:\Users\Public\Documents\Equivital\Equivital ...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,DATA,Pilot_3,3420427,230208,0,DATA-Pilot_3-3420427-23020800.CSV,csv,109,2020-02-02 02:02:00+00:00,C:\Users\Public\Documents\Equivital\Equivital ...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,DATA,Pilot_3,3420427,230210,0,DATA-Pilot_3-3420427-23021000.CSV,csv,134,2020-02-02 02:02:00+00:00,C:\Users\Public\Documents\Equivital\Equivital ...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,DATA,Pilot_30,5022141,230130,0,DATA-Pilot_30-5022141-23013000.CSV,csv,134,2020-02-02 02:02:00+00:00,C:\Users\Public\Documents\Equivital\Equivital ...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
217,DATA,LS410,5022154,230213,0,DATA-LS410-5022154-23021300.CSV,csv,134,2020-02-02 02:02:00+00:00,C:\Users\Public\Documents\Equivital\Equivital ...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
218,DATA,LS410,5022154,230213,101,DATA-LS410-5022154-23021300_0.CSV,csv,134,2020-02-02 02:02:00+00:00,C:\Users\Public\Documents\Equivital\Equivital ...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
219,DATA,Pilot 56,5022163,230131,0,DATA-Pilot 56-5022163-23013100.CSV,csv,134,2020-02-02 02:02:00+00:00,C:\Users\Public\Documents\Equivital\Equivital ...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
220,DATA,Pilot 55,5022131,230131,0,DATA-Pilot 55-5022131-23013100.CSV,csv,134,2020-02-02 02:02:00+00:00,C:\Users\Public\Documents\Equivital\Equivital ...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


These recordings can be checked in View_Equivital_Qiosk_Export.

Now we move them

# Move selected files to suitable project folder

In [70]:
projectsFolder = 'C:\\Users\\fourMs lab\\Documents\\Equivital\\ProjectData\\' # where qiosk files are moved to after export
print(path) # where the qiosk files are initially saved by the Equivital Qiosk program

C:\Users\Public\Documents\Equivital\Equivital Manager Wizard\


In [71]:
# set project folder name
projectName = 'AlexanderStandStill'
#projectName = 'BatteryTests'
#projectName = 'SoloRecordings'
#projectName = 'Concert230128'
#projectName = 'LauraTestRecording'
projectPath = projectsFolder + projectName + '\\'

### Create folders for project to move files to
Check target location for data files to keep and to remove

In [72]:
# check if the folder exists for project and set up locations
if not os.path.isdir(projectsFolder + projectName):
    os.mkdir(projectsFolder + projectName)
    
if not os.path.isdir(projectsFolder + projectName + '\\' + 'SEM'):
    os.mkdir(projectsFolder + projectName + '\\' + 'SEM')
if not os.path.isdir(projectsFolder + projectName + '\\' + 'CSV'):
    os.mkdir(projectsFolder + projectName + '\\' + 'CSV')

### Move CSV and SEM files to project folder
Move them to project folder and generate their own present recordings file.

In [73]:
for i,row in s_files.iterrows():
    dataFile = row['FullLoc']
    matched = qex.matched_files(dataFile,path) # outputs locations of csv and sem files
    
    for fi in matched:
        fileName = fi.split('\\')[-1]
        devName = row['DevName']
        if fileName.lower().endswith('csv'):
            if not fileName.lower().endswith('Recordings.csv'):
                out_f = projectPath + 'CSV\\' + fileName
                os.rename(fi,out_f)
        if fileName.lower().endswith('sem'):
            if not os.path.isdir(projectsFolder + projectName + '\\SEM\\' + devName):
                os.mkdir(projectsFolder + projectName  + '\\SEM\\' + devName)
            out_f = projectPath + 'SEM\\' + devName  + '\\' + fileName
            os.rename(fi,out_f)
            
dfiles = qex.qiosk_recordings(projectPath,projectName)
dfiles

,Signal,DevName,ID,Date,Session,FileName,FileType,FileSize,RecStart,RecEnd,...,SubjectNames,SENSOR ID,SUBJECT ID,SUBJECT AGE,HR(BPM),HRC(%),BELT OFF,LEAD OFF,MOTION,BODY POSITION
0,DATA,Pilot_1,3420755,230131,0,DATA-Pilot_1-3420755-23013100.CSV,csv,12267,2023-01-31 14:38:04+00:00,2023-01-31 15:15:49+00:00,...,[Pilot_1],3420755.0,Pilot_1,NaN,79,95.0,0.0,1.0,Stationary,Upright
1,DATA,Pilot_1,3420755,230201,101,DATA-Pilot_1-3420755-23020100_0.CSV,csv,5366,2023-02-01 12:01:29+00:00,2023-02-01 12:17:44+00:00,...,[Pilot_1],3420755.0,Pilot_1,NaN,84,100.0,0.0,0.0,Stationary,Upright
2,DATA,Pilot_2,3420456,230203,0,DATA-Pilot_2-3420456-23020300.CSV,csv,8532,2023-02-03 11:40:06+00:00,2023-02-03 12:06:36+00:00,...,[Pilot_2],3420456.0,Pilot_2,NaN,0,99.0,0.0,0.0,Stationary,Upright


# Clear Recordings to ToBeDeleted

In [10]:
# if these are files to be discarded store the DATA file indexes to discardRecs
discardRecs = s_files.index
projTag = 'ToBeDiscarded'
discardPath = 'C:\\Users\\fourMs lab\\Documents\\Equivital\\ProjectData\\ToBeDeleted\\'

for i,row in s_files.iterrows():
    dataFile = row['FullLoc']
    matched = qex.matched_files(dataFile,path,seperator) # outputs locations of csv and sem files
    
    for f in matched:
        fileName = f.split('\\')[-1]
        devName = row['DevName']
        if f.lower().endswith('csv'):
            out_f = discardPath + 'CSV\\' + fileName
            shutil.move(f,out_f)
        if f.lower().endswith('sem'):
            if not os.path.isdir(discardPath + 'SEM\\' + devName):
                os.mkdir(discardPath + 'SEM\\' + devName)
            out_f = discardPath + 'SEM\\' + devName + '\\' + fileName
            shutil.move(f,out_f)
            


TypeError: qiosk_recordings() missing 1 required positional argument: 'sep'

In [11]:
dfiles = qex.qiosk_recordings(discardPath,projTag,seperator)
print(len(dfiles))

585


# if Files need to be moved between project folders

Set the origin project name and target project name, collect the existing files at that location, chose the subset of recordings to move or copy, move or copy them, generate recording list in each location

In [40]:
# incase files need to be shifted from a project folder
projectName = 'SoloRecordings'
projectPath = projectsFolder + projectName + '\\'
projectPath1 = projectPath
projectName1 = projectName

s_files = qex.qiosk_recordings(projectPath1,projectName1)
s_files 

,Signal,DevName,ID,Date,Session,FileName,FileType,FileSize,RecStart,RecEnd,...,SubjectNames,SENSOR ID,SUBJECT ID,SUBJECT AGE,HR(BPM),HRC(%),BELT OFF,LEAD OFF,MOTION,BODY POSITION
0,DATA,Test1,3420415,230120,7,DATA-Test1-3420415-23012007.CSV,csv,858075,2023-01-20 22:00:08+00:00,2023-01-21 13:31:13+00:00,...,[Test1],3420415.0,Test1,NaN,61,99,0,0,MovingSlowly,Side
1,DATA,Pilot_1,3420755,230201,102,DATA-Pilot_1-3420755-23020100_1.CSV,csv,62494,2023-02-01 17:14:56+00:00,2023-02-01 20:30:11+00:00,...,[Pilot_1],3420755.0,Pilot_1,NaN,78,98,0,0,Stationary,Upright
2,DATA,Bluetooth_Tester_1,3420415,230203,0,DATA-Bluetooth_Tester_1-3420415-23020300.CSV,csv,196,2023-02-03 10:10:03+00:00,2023-02-03 10:10:03+00:00,...,[Bluetooth_Tester_1],3420415.0,Bluetooth_Tester_1,NaN,30,97,1,1,Stationary,Side


In [41]:
# get subset of s_files as you see fit
s_files = s_files.loc[2:2,:].copy()
s_files

,Signal,DevName,ID,Date,Session,FileName,FileType,FileSize,RecStart,RecEnd,...,SubjectNames,SENSOR ID,SUBJECT ID,SUBJECT AGE,HR(BPM),HRC(%),BELT OFF,LEAD OFF,MOTION,BODY POSITION
2,DATA,Bluetooth_Tester_1,3420415,230203,0,DATA-Bluetooth_Tester_1-3420415-23020300.CSV,csv,196,2023-02-03 10:10:03+00:00,2023-02-03 10:10:03+00:00,...,[Bluetooth_Tester_1],3420415.0,Bluetooth_Tester_1,NaN,30,97,1,1,Stationary,Side


In [42]:
# project these files should be moved to
projectName = 'ToBeDeleted'
# check if the folder exists for project and set up locations
if not os.path.isdir(projectsFolder + projectName):
    os.mkdir(projectsFolder + projectName)
    
if not os.path.isdir(projectsFolder + projectName + '\\' + 'SEM'):
    os.mkdir(projectsFolder + projectName + '\\' + 'SEM')
if not os.path.isdir(projectsFolder + projectName + '\\' + 'CSV'):
    os.mkdir(projectsFolder + projectName + '\\' + 'CSV')
    
projectPath = projectsFolder + projectName + '\\'
projectPath2 = projectPath
projectName2 = projectName

In [43]:
print(row['FullLoc'])
matched = qex.matched_files(dataFile,projectPath1)

C:\Users\fourMs lab\Documents\Equivital\ProjectData\SoloRecordings\CSV\DATA-Bluetooth_Tester_1-3420415-23020300.CSV


In [54]:
# move s_files
for i,row in s_files.iterrows():
    dataFile = row['FullLoc']
    matched = qex.matched_files(dataFile,projectPath1) # outputs locations of csv and sem files
    
    for fi in matched:
        fileName = fi.split('\\')[-1]
        devName = row['DevName']
        if fileName.lower().endswith('csv'):
            if not fileName.lower().endswith('Recordings.csv'):
                out_f = projectPath2 + 'CSV\\' + fileName
                shutil.move(fi,out_f)
        if fileName.lower().endswith('sem'):
            if not os.path.isdir(projectPath2 + 'SEM\\' + devName):
                os.mkdir(projectPath2 + 'SEM\\' + devName)
            out_f = projectPath2 + 'SEM\\' + devName  + '\\' + fileName
            shutil.move(fi,out_f)

In [49]:
# COPY s_files
for i,row in s_files.iterrows():
    dataFile = row['FullLoc']
    matched = qex.matched_files(dataFile,projectPath1) # outputs locations of csv and sem files
    
    for fi in matched:
        fileName = fi.split('\\')[-1]
        devName = row['DevName']
        if fileName.lower().endswith('csv'):
            if not fileName.lower().endswith('Recordings.csv'):
                out_f = projectPath2 + 'CSV\\' + fileName
                #os.system('cp ' + fi + ' ' + out_f)
                shutil.copy2(fi,out_f) # complete target filename given

        if fileName.lower().endswith('sem'):
            if not os.path.isdir(projectPath2 + 'SEM\\' + devName):
                os.mkdir(projectPath2 + 'SEM\\' + devName)
            out_f = projectPath2 + 'SEM\\' + devName  + '\\' + fileName
            shutil.copy2(fi,out_f) # complete target filename given


In [50]:
fi

'C:\\Users\\fourMs lab\\Documents\\Equivital\\ProjectData\\SoloRecordings\\SEM\\Bluetooth_Tester_1\\23020300.SEM'

In [51]:
print(projectPath2)
out_f

C:\Users\fourMs lab\Documents\Equivital\ProjectData\ToBeDeleted\


'C:\\Users\\fourMs lab\\Documents\\Equivital\\ProjectData\\ToBeDeleted\\SEM\\Bluetooth_Tester_1\\23020300.SEM'

In [55]:
dfiles = qex.qiosk_recordings(projectPath1,projectName1)
print(projectPath1+projectName1)
print(dfiles)
dfiles = qex.qiosk_recordings(projectPath2,projectName2)
print(projectPath2+projectName2)
print(dfiles)

C:\Users\fourMs lab\Documents\Equivital\ProjectData\SoloRecordings\SoloRecordings
  Signal  DevName       ID    Date  Session  \
0   DATA    Test1  3420415  230120        7   
1   DATA  Pilot_1  3420755  230201      102   

                              FileName FileType  FileSize  \
0      DATA-Test1-3420415-23012007.CSV      csv    858075   
1  DATA-Pilot_1-3420755-23020100_1.CSV      csv     62494   

                   RecStart                    RecEnd  ...  SubjectNames  \
0 2023-01-20 22:00:08+00:00 2023-01-21 13:31:13+00:00  ...       [Test1]   
1 2023-02-01 17:14:56+00:00 2023-02-01 20:30:11+00:00  ...     [Pilot_1]   

   SENSOR ID  SUBJECT ID  SUBJECT AGE HR(BPM) HRC(%)  BELT OFF LEAD OFF  \
0  3420415.0       Test1          NaN      61     99         0        0   
1  3420755.0     Pilot_1          NaN      78     98         0        0   

         MOTION  BODY POSITION  
0  MovingSlowly           Side  
1    Stationary        Upright  

[2 rows x 25 columns]
C:\Users\fourMs